# Architecture A — single-agent baseline

Questa notebook prepara l'ambiente su Colab, clona la repo, configura i modelli HuggingFace e lancia il grafo per l'architettura **A** (single-agent → tester). Salva i log in `log/`.

Note operative:
- Imposta un token HuggingFace valido (HF Inference API) quando richiesto.
- Lancia pochi task della APPS per evitare costi/tempo eccessivi.
- I log sono salvati sia su stdout sia in file JSONL/LOG per analisi successive.

In [1]:
import os
import sys
import subprocess
import pathlib

REPO_URL = "https://github.com/LLM4SE-group-15/ArchitecturesForCodeDevelopmentWithLLMs.git"
REPO_DIR = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

os.chdir(REPO_DIR)
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "pip"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)

print(f"Using repo at {REPO_DIR.resolve()}")

Using repo at /content/ArchitecturesForCodeDevelopmentWithLLMs


In [2]:
!pip install -r requirements.txt

In [3]:
import os
import getpass
from huggingface_hub import login

# Forza sempre l'uso del token che inserisci
os.environ["HF_TOKEN"] = getpass.getpass("Enter HF_TOKEN (kept hidden): ")

login(token=os.environ["HF_TOKEN"], add_to_git_credential=False)

os.environ["ARCHITECTURE"] = "A"
print("ARCHITECTURE set to", os.environ["ARCHITECTURE"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


ARCHITECTURE set to A


In [4]:
from huggingface_hub import HfApi

api = HfApi()
try:
    user_info = api.whoami(token=os.environ["HF_TOKEN"])
    print("Logged in to Hugging Face as:", user_info.get("name") or user_info.get("user"))
except Exception as exc:
    print("Login check failed:", exc)

Logged in to Hugging Face as: Riaburger


In [5]:
import json
import time
import logging
import pathlib
import os

from datetime import datetime

DEFAULT_ROOT = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")
ROOT = DEFAULT_ROOT if DEFAULT_ROOT.exists() else pathlib.Path.cwd()

sys.path.insert(0, str(ROOT))

LOG_DIR = ROOT / "log"
LOG_DIR.mkdir(exist_ok=True)

logger = logging.getLogger("architecture_A")
logger.setLevel(logging.INFO)
if logger.handlers:
    logger.handlers.clear()
formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
file_handler = logging.FileHandler(LOG_DIR / "architecture_A.log")
stream_handler = logging.StreamHandler()
for handler in (file_handler, stream_handler):
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Logger ready. Repo root: %s", ROOT)
logger.info("Log files: %s", LOG_DIR)
print("Logs ->", LOG_DIR)

2026-01-03 17:18:33,134 | INFO | Logger ready. Repo root: /content/ArchitecturesForCodeDevelopmentWithLLMs
INFO:architecture_A:Logger ready. Repo root: /content/ArchitecturesForCodeDevelopmentWithLLMs
2026-01-03 17:18:33,136 | INFO | Log files: /content/ArchitecturesForCodeDevelopmentWithLLMs/log
INFO:architecture_A:Log files: /content/ArchitecturesForCodeDevelopmentWithLLMs/log


Logs -> /content/ArchitecturesForCodeDevelopmentWithLLMs/log


In [6]:
import time
from src.data.task_loader import APPSTaskLoader
from src.graph.graph import run_graph
from src.agents.llm import Architecture

ARCH = Architecture.A

def run_sample_tasks(
    per_level: int = 10,
    split: str = "test",
    difficulties: tuple[str, ...] = ("introductory",),
    shuffle: bool = True,
):
    loader = APPSTaskLoader(split=split)
    if shuffle:
        loader._dataset = loader.dataset.shuffle(seed=int(time.time()))

    tasks = []
    for diff in difficulties:
        tasks.extend(loader.load_by_difficulty(diff, limit=per_level))

    results = []
    total = len(tasks)
    logger.info("Loaded %s tasks (%s per difficulty: %s)", total, per_level, ", ".join(difficulties))

    for idx, task in enumerate(tasks, 1):
        logger.info("Running %s/%s %s (%s)", idx, total, task.task_id, task.difficulty)
        start = time.time()
        state = run_graph(
            task_id=task.task_id,
            task_description=task.question,
            test_inputs=task.inputs,
            test_outputs=task.outputs,
            architecture=ARCH,
        )

        print("State:", state)
        print()

        elapsed = time.time() - start
        record = {
            "task_id": task.task_id,
            "difficulty": task.difficulty,
            "architecture": str(ARCH.value),
            "test_passed": state["test_passed"],
            "developer_tier": state.get("developer_tier"),
            "escalations": state["escalations"],
            "story_points_initial": state.get("story_points_initial"),
            "story_points_final": state.get("story_points_current"),
            "elapsed_seconds": elapsed,
        }
        results.append(record)
        logger.info(
            "Finished %s | pass=%s tier=%s escalations=%s elapsed=%.1fs",
            task.task_id,
            state["test_passed"],
            record["developer_tier"],
            record["escalations"],
            elapsed,
        )
        with open(LOG_DIR / "architecture_A.jsonl", "a", encoding="utf-8") as f:
            f.write(json.dumps(record) + "\n")
    return results

sample_results = run_sample_tasks(per_level=10, difficulties=("introductory",), shuffle=True)
sample_results

Loading APPS dataset (test split)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

2026-01-03 17:19:22,724 | INFO | Loaded 10 tasks (10 per difficulty: introductory)
INFO:architecture_A:Loaded 10 tasks (10 per difficulty: introductory)
2026-01-03 17:19:22,725 | INFO | Running 1/10 apps_4590 (introductory)
INFO:architecture_A:Running 1/10 apps_4590 (introductory)


...

2026-01-03 17:20:03,572 | INFO | Finished apps_4823 | pass=False tier=None escalations=0 elapsed=3.5s
INFO:architecture_A:Finished apps_4823 | pass=False tier=None escalations=0 elapsed=3.5s
2026-01-03 17:20:03,574 | INFO | Running 10/10 apps_4048 (introductory)
INFO:architecture_A:Running 10/10 apps_4048 (introductory)


State: {'task_id': 'apps_4823', 'task_description': 'You are given a five-card hand drawn from a standard $52$-card deck. The strength of your hand is the maximum value $k$ such that there are $k$ cards in your hand that have the same rank.\n\nCompute the strength of your hand.\n\n-----Input-----\nThe input will consist of a single line, with five two-character strings separated by spaces.\n\nThe first character in each string will be the rank of the card, and will be one of A23456789TJQK. The second character in the string will be the suit of the card, and will be one of CDHS.\n\nYou may assume all the strings are distinct.\n\n-----Output-----\nOutput, on a single line, the strength of your hand.\n\n-----Examples-----\nSample Input 1:\nAC AD AH AS KD\nSample Output 1:\n4\n\nSample Input 2:\n2C 4D 4H 2D 2H\nSample Output 2:\n3', 'plan': None, 'story_points_initial': None, 'story_points_current': None, 'escalations': 0, 'developer_tier': None, 'generated_code': "import collections\n\nde

2026-01-03 17:20:10,903 | INFO | Finished apps_4048 | pass=False tier=None escalations=0 elapsed=7.3s
INFO:architecture_A:Finished apps_4048 | pass=False tier=None escalations=0 elapsed=7.3s


State: {'task_id': 'apps_4048', 'task_description': 'Takahashi is standing on a multiplication table with infinitely many rows and columns.\nThe square (i,j) contains the integer i \\times j. Initially, Takahashi is standing at (1,1).\nIn one move, he can move from (i,j) to either (i+1,j) or (i,j+1).\nGiven an integer N, find the minimum number of moves needed to reach a square that contains N.\n\n-----Constraints-----\n - 2 \\leq N \\leq 10^{12}\n - N is an integer.\n\n-----Input-----\nInput is given from Standard Input in the following format:\nN\n\n-----Output-----\nPrint the minimum number of moves needed to reach a square that contains the integer N.\n\n-----Sample Input-----\n10\n\n-----Sample Output-----\n5\n\n(2,5) can be reached in five moves. We cannot reach a square that contains 10 in less than five moves.', 'plan': None, 'story_points_initial': None, 'story_points_current': None, 'escalations': 0, 'developer_tier': None, 'generated_code': 'import math\n\ndef min_moves_to_r

[{'task_id': 'apps_4590',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 5.587212800979614},
 {'task_id': 'apps_4573',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 7.5999555587768555},
 {'task_id': 'apps_4713',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': True,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 2.982330083847046},
 {'task_id': 'apps_4241',
  'difficulty': 'introductory',
  'architecture': 'A',
  'test_passed': False,
  'developer_tier': None,
  'escalations': 0,
  'story_points_initial': None,
  'story_points_final': None,
  'elapsed_seconds': 6.56

In [39]:
!cd log && cat architecture_A.jsonl

{"task_id": "apps_4000", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 21.43400001525879}
{"task_id": "apps_4000", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 24.04499888420105}
{"task_id": "apps_4406", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 2.9327504634857178}
{"task_id": "apps_4326", "difficulty": "introductory", "architecture": "A", "test_passed": false, "developer_tier": null, "escalations": 0, "story_points_initial": null, "story_points_final": null, "elapsed_seconds": 2.3970892429351807}
{"task_id": "apps_4118", "difficulty": "introductory",

Miao